# Qué dicen los datos sobre los festivales de Barcelona
 ### Análisis sobre los eventos artísticos más populares entre 2013 - 2022

## Índice:<br>

1. Introducción<br>
2. Aclaraciones iniciales
3. Importación de bibliotecas
4. Importación y limpieza de datos
5. Hipótesis: <br>
* El sector público programa festivales de ámbitos desatendidos por los programadores privados <br>
* La mayoría de los festivales se concentra justo antes y después de los meses de calor y playa <br>
* Los festivales gratuitos son los que convocan mayor número de asistentes <br>
* En Barcelona ha crecido la tendencia a programar macrofestivales <br>
* La pandemia marcó un quiebre en el tipo de programación, que ya se ha superado <br>

#### Introducción:

En este EDA he buscado hacer una aproximación al estudio de la oferta d fetsivales artísticos en Barcelona, atendiendo a sus ámbitos artísticos, cantidad de asistentes, la época del año en que tienen lugar, así como si son organizados por organismos públicos o empresas privados, si cobran o no entrada, entre otras cosas.<br><br>
Mediante el análisis de datos he buscado entender qué diferencias hay entre los festivales programados por la administración pública frente a los gestionados por iniciativas privadas, así como entender la distribución de los eventos a lo largo del año, y ver posibles relaciones entre esta distribución y los ámbitos artísticos.<br><br>
Otra inquietud ha sido indagar si los fetsivales con entrada de pago presentan una barrera de acceso que influya en el número de asistentes, o si es en ciertos sectores culturales donde hay mayor hábito de pagar por las entradas, frente a otros sectores con mayor tendencia a los eventos gratuitos.<br><br>
Finalmente, he querido entender cómo ha sido la evolución de los festivales a lo largo de una década, con el ojo puesto en las tendencias, tanto de ámbito artístico como de la envergadura de los propios festivales. En esta evolución también me ha parecido interesante atender a los posibles cambios que se desataron por la pandemia, y en qué medida éstos se han mantendo o han vuelto a su cauce inicial pasados un par de años.<br><br>

##### Fuentes de datos

Los datos que he usado para el presente EDA son de carácter público, y están disponibles en una serie de webs de las administraciones públicas, tanto del [Ajuntamiento de Barcelona](https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals), de donde los he tomado, como de las administraciones que la engloban a nivel autonómico y nacional.

#### Aclaraciones iniciales

Creo importante destacar el recorte conceptual a la hora de incluir, o no, determinados eventos incluidos en los datos recogidos.<br>
En primer lugar, he dejado fuera aquellas entradas que estaban clasificadas como ferias. <br>
Y en segundo, dentro delas clasificadas como festivales, he hecho también un recorte a favor de aquellos eventos considerdos festivales en el sentido más próximo a fiesta, y descartando, en cambio, aquellos que por el contrario se acercan más a conferencias o mesas redondas. En consecuencia, una buena parte de los eventos catalogados en la base de datos como fetsivales de Letras han quedado fuera.<br>
También he decidido quitar aquellos eventos que por el formato eran solo ciclos, muestras o encuentros de pequeño formato.<<br>><br>
En una segunda instancia, he realizado un filtro tomando de cada uno de los 10 años de muestra aquellos 50 festivales con mayor número de asistentes y que tuvieran al menos 3 edicionesdentro del período analizado, para evitar incluir casos aislados que pudieran distorcionar la muestra y las conclusiones.

#### Importación de bibliotecas

Para llevar adelante este EDA he recurrido a las siguientes bibliotecas:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Importación de datos
Y estas son las tablas que he usado como punto de partida para mi EDA:

In [3]:
fests_22 = pd.read_csv("./data/2022_festivals-assistents-order-name.csv")
fuente_1 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/c573360d-ed5d-4609-aaa5-55bb9e9175ca#additional-info"
fests_21 = pd.read_csv("./data/2021_festivals-assistents-order-name.csv")
fuente_2 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/57ed9976-8291-418d-a0fc-ecf572d8cb74?inner_span=True"
fests_20= pd.read_csv("./data/2020_festivals-assistents-order-name.csv")
fuente_3 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/1214655c-38e5-4434-a35c-ed3a6468cd23?inner_span=True"
fests_19 = pd.read_csv("./data/2019_festivals-assistents-order-name.csv")
fuente_4 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/077d738b-997f-46b9-ab96-500ab22f32b6?inner_span=True"
fests_18 = pd.read_csv("./data/2018_festivals-assistents-order-name.csv")
fuente_5 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/a5522295-70f1-4d0b-8cb2-8762cbcbf783?inner_span=True"
fests_17 = pd.read_csv("./data/2017_festivals-assistents.csv")
fuente_6 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/7760d11f-a95f-4d6c-8230-e409b8ce6995?inner_span=True"
fests_16 = pd.read_csv("./data/2016_festivals-assistents.csv")
fuente_7 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/d151b7bc-2cb4-48c9-89e3-1c733a7cf511?inner_span=True"
fests_15 = pd.read_csv("./data/2015_festivals-assistents.csv")
fuente_8 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/5797a47d-45c9-40c3-aacb-2d3fcd5c6cf7?inner_span=True"
fests_14 = pd.read_csv("./data/2014_festivals-assistents.csv")
fuente_9 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/f2afdc37-2d91-48e5-9319-4d94d57d8c63?inner_span=True"
fests_13 = pd.read_csv("./data/2013_festivals-assistents.csv")
fuente_10 = "https://opendata-ajuntament.barcelona.cat/data/es/dataset/dades-festivals/resource/ae5be150-db8f-4d66-8605-78bf2f02d375?inner_span=True"

#### Limpieza de datos

Luego, he procesado cada una de esas tablas con una serie de funciones para añadir, transformar y eliminar filas, columnas y valores, como se puede observar en el correspondiente notebook 1_preparacion_datos <BR> <BR>AÑADIR ENLACE!!!!<BR><BR>
Una vez realizado este primer paso de limpieza general, he pasado al análisis individual de cada dataframe, cada uno correspondiente a uno de los años de la década analizada, y sobre ellos he realizado prácticamente las mismas opraciones, que luego me permitieran compararlos y encontrar similitudes y diferencia. <r>
Luego, he procesado cada una de esas tablas con una serie de funciones para añadir, transformar y eliminar filas, columnas y valores, como se puede observar en el correspondiente notebook 1_preparacion_datos <BR> <BR>AÑADIR ENLACE!!!!<BR><BR>
De este modo, resulta una colección de 10 notebooks, uno por año para analizar la situación de los festivales en Barcelona en los años 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021 y 2022. <BR> <BR>AÑADIR ENLACE!!!!<BR><BR> La organización del análisis univariable de cada dataframe / año en un notebook diferente ha hecho más sencillo su anñalisis, comprensión y posterior comparación.<br>
Posteriormente, en un nuevo notebook, he procedido al análisis multivariante, en este caso organizado según criterios de comparación, listando la misma operación para cada uno de los años, con la intencion de poner ahora el énfasis en la evolución de la escena en conjunto.
<BR> <BR>AÑADIR ENLACE!!!!<BR><BR>
Lo que sigue a continuación es un extacto de los principales pasos y análisis orientados a corroborar o descatar las hipótesis del siguiente EDA